In [ ]:
"""
 Befor installing packages on anaconda it's better to create an environment such as:
 conda create --name graph_env python=3.10 -y
 conda activate graph_env

 
 Using following command will install some necessary packages on anaconda:
 conda install -y matplotlib=3.5.0 scikit-learn networkx=2.6.3 pytorch torchvision torchaudio -c pytorch

 In this environment the following versions of packages are installed:
 python=3.10
 pytorch=2.5.1
 scikit-learn=1.6.1
 numpy=1.26.4
 torch_geometric=2.6.1
 matplotlib=3.5.0
 karateclub=1.3.3

 After installation of pytorch you should install the following packages as well: 
 Pay attention to: torch-2.5.1+cpu: version of installed torch and we just using CPU
 pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.1+cpu.html

 We should also install the following package:
 pip install torch_geometric

 for the installation of karateclub: 
 pip install karateclub==1.3.3 --no-deps
 
 You may needs to install the following packages:
 pip install numpy<1.23.0 pandas<=1.3.5 decorator==4.4.2 gensim>=4.0.0 pygsp python-Levenshtein
 pip install gensim>=4.0.0
 pip install pygsp
 pip install "pandas<=1.3.5"
 pip install ipywidgets
 
 If you want to uninstall the created environment you can use the following commands:
 conda deactivate
 conda remove --name graph_env --all

 conda clean --all -y  # Clears cache and temporary files
 conda update -n base conda -y  # Updates Conda itself
 """

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import sklearn 
print(sklearn.__version__)

In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
import torch_geometric
print(torch_geometric.__version__)

In [ ]:
import matplotlib
print(matplotlib.__version__)

In [ ]:
import karateclub
print(karateclub.__version__)

In [ ]:
# At first we should load a graph.
# We used Zachary's karate club.
import networkx as nx
graph_karate_club = nx.karate_club_graph()

In [ ]:
# 1. Basic Graph Information
print("=== Basic Graph Information ===")
print(f"Number of nodes: {graph_karate_club.number_of_nodes()}") #34 vertexes
print(f"Number of edges: {graph_karate_club.number_of_edges()}") # 78 edges (friendship)

In [ ]:
# 2. Get the community structure of graph.
clubs = nx.get_node_attributes(graph_karate_club, 'club')

print("Attributes of graph (nodes and their affiliations): \n", clubs)
print("\nType of clubs is: ",type(clubs))
# We change type of 'clubs from dictionary to set and print.
# In set we don't have repetitive values. 
print(f"\n\n Two types of attributes that each node has: {set(clubs.values())}")  # {'Mr. Hi', 'Officer'} 
print(f"Mr. Hi's faction size: {list(clubs.values()).count('Mr. Hi')}") 
print(f"Officer's faction size: {list(clubs.values()).count('Officer')}") 

In [ ]:
# Now we want to chack some basic informatio of a vertex in graph_karate_club
# Choose a node to inspect (e.g., node 0 = instructor, node 33 = administrator)
node_id = 0 
# 1. Get basic node information
print(f"--- Node {node_id} Properties ---")
print(f"Degree: {graph_karate_club.degree[node_id]}")  # Number of connections
print(f"Club affiliation: {graph_karate_club.nodes[node_id]['club']}")  # 'Mr. Hi' or 'Officer'

In [ ]:
# 2. Get neighbors (directly connected nodes)
neighbors = list(graph_karate_club.neighbors(node_id))
print(f"Neighbors: {neighbors}")

In [ ]:
# Convert labels of nodes into 0 and 1 and store them in a list.
labels = [1 if graph_karate_club.nodes[i]['club'] == 'Officer' else 0 for i in graph_karate_club.nodes()]
print(f"labels in the for of 0 and 1 is: {labels}")

In [ ]:
#Plot the Graph
layout_pos = nx.spring_layout(graph_karate_club)
nx.draw_networkx(graph_karate_club, pos = layout_pos, node_color=labels, cmap='coolwarm')

In [ ]:
# 3. Visualize local connections
import matplotlib.pyplot as plt

#Highlight the target node and its connections
node_colors = ['red' if i == node_id else 'skyblue' for i in graph_karate_club.nodes()]
node_sizes = [300 if i == node_id else 100 for i in graph_karate_club.nodes()]

nx.draw(graph_karate_club, 
        with_labels=True,
        node_color=node_colors,
        node_size=node_sizes,
        font_size=8)

plt.title(f"Node {node_id} (Red) and its connections")
plt.show()

In [ ]:
# 1. Initialize DeepWalk model
from karateclub import DeepWalk
DeepWalkModel = DeepWalk(walk_number=10, walk_length=40, dimensions=128)

In [ ]:
# 2. Fit the model to the graph
DeepWalkModel.fit(graph_karate_club)

In [ ]:
# 3. Fit the model to the graph
embedding = DeepWalkModel.get_embedding()
# We have 34 vertexes and in the creation of DeepWalk model we used 128 dimenstion for embedding of each vertex 
print('The embedding model has the following shape: ', embedding.shape)

In [ ]:
# 4. Visualize embeddings (2D PCA)
import sklearn

pca = sklearn.decomposition.PCA(n_components=2)
emb_2d = pca.fit_transform(embedding)

In [ ]:
# Color nodes by their ground-truth 
import matplotlib.pyplot as plt

colors = ['red' if graph_karate_club.nodes[i]['club'] == 'Mr. Hi' else 'blue' for i in graph_karate_club.nodes()]
plt.figure(figsize=(8, 6))
plt.scatter(emb_2d[:, 0], emb_2d[:, 1], c=colors, alpha=0.7)
plt.title("DeepWalk Embeddings of Karate Club Network")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")

In [ ]:
# 5. Split data into train/test sets (80/20)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    embedding,
    labels, 
    test_size=0.2,
    random_state = 42,      
)

In [ ]:
# 6. Train a classifier
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

In [ ]:
# 6. Evaluate performance
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_preds = clf.predict(X_test)

print(f"Verified Test Accuracy: {accuracy_score(y_test, y_preds):.2f}\n")

print("Classification Report:")
print(classification_report(y_test, y_preds))